In [1]:
import logging
import couchdb
import json
import sys
sys.path.append('./release')
import sentiment_prediction as senti

Using TensorFlow backend.


In [2]:
'''
    Get the source db(the db created by harverster)
    and the destination db (store the result)
'''
def get_db():
    config = None
    server = None
    db = None
    with open("db_config.json") as cf:
        config = json.load(cf)
        try:
            db_server = couchdb.Server(config["Servers"][0])
            source_db_name = config["DB"][0]
            dest_db_name = config["DB"][1]
            if dest_db_name in db_server:
                logging.info("Database %s has existed, using the original one." %(dest_db_name))
                dest_db = db_server[dest_db_name]
            else:
                logging.info("Database %s does not exist, creating a new one." %(dest_db_name))
                dest_db = db_server.create(dest_db_name)
            if source_db_name not in db_server:
                logging.error("Database %s does not exist, something went wrong." %(source_db_name))
                return
            source_db = db_server[source_db_name]
        except Exception as e:
            logging.error("error happened: %s" %e)
            sys.exit(2)
    return source_db,dest_db

In [3]:
####################This is test part######################
# import csv
# source_db,dest_db = get_db()
# csv_file = csv.reader(open("sentiment140-small.csv"))
# d = []
# for i in csv_file:
#     d.append({"time":i[2], "raw_tweet":i[5]})
# source_db.update(d)

In [3]:
'''
    This is the excution part, the analysis result will be
    stored into the dest db
'''
def analyze(source_db,dest_db,modelpath,pklpath,mode="Sentiment"):
    analysor = None
    #more modes can be added here#
    if mode == "Sentiment":
        analysor = senti.sentianalysor(modelpath, pklpath, stemmer = False)
        count = 0
        result = []
        for ele in source_db:
            count += 1
            time = source_db[ele]["time"]
            raw_tweet = source_db[ele]["raw_tweet"]
            label,proba = analysor.prediction(raw_tweet)
            #0:NEGATIVE 1:POSITIVE
            label_flag = 0 if label=="NEGATIVE" else 1
            result.append({"time":time,"raw_tweet":raw_tweet,"label":label_flag,"probability":proba})
            if count == 100:
                dest_db.update(result)
                count = 0
                result = []
        if count != 0 and len(result) != 0:
            dest_db.update(result)
        print("Analyze successfully.")

In [4]:
if __name__ == "__main__":
    modelpath = './model/sentiment_lstm.h5'
    pklpath = './model/sentiment140-freqdist.pkl'
    source_db,dest_db = get_db()
    analyze(source_db,dest_db,modelpath,pklpath)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Analyze successfully.
